In [53]:
import re
import json
import os
import chardet
from pathlib import Path
from typing import List
from autogen import Agent, AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager, ConversableAgent, register_function
from autogen.coding import LocalCommandLineCodeExecutor
import ollama
import fix_busted_json
import pandas as pd
from ragFunction import * 


In [54]:
# 檔案
MASTER_JS_PATH = r"C:\users\tuf\documents\cloudysys_nickfury\dcmsln\dcmsln_201812\Master\PageJS\Query"
js_files = list(Path(MASTER_JS_PATH).rglob("*.js"))

if not js_files:
    raise SystemExit("找不到 JS 檔案")

# decoder 
def read_text_file_safely(path: str) -> str:
    with open(path, "rb") as f:
        raw_data = f.read()
        detected = chardet.detect(raw_data)
        encoding = detected["encoding"] or "utf-8"

    fallback_encodings = []

    if encoding: # Add the detected encoding first
        fallback_encodings.append(encoding)
    if 'GB2312' not in fallback_encodings: # Add GB2312 if not already present
        fallback_encodings.append("GB2312")

    # Ensure common encodings are covered
    additional_encodings = ["utf-8", "big5", "cp950", "gbk", "gb18030", "utf-16", "windows-1252"]
    for enc in additional_encodings:
        if enc not in fallback_encodings:
            fallback_encodings.append(enc)

    for enc in fallback_encodings:
        try:
            decoded = raw_data.decode(enc)

            # 這裡移除常見控制字符、零寬度字元：換行、tab、回車等
            # \s 匹配所有空白字元，包括傳統空白和許多 Unicode 空白字元
            cleaned = re.sub(r'[\uFEFF\u200B\r\n\t]', '', decoded)
            
            # 去掉多餘空白並移除前後空白
            cleaned = cleaned.strip()
            return cleaned
        except UnicodeDecodeError:
            continue

    raise Exception(f"所有常見編碼皆無法解碼檔案: {path}")

# 切分 function
def extract_function_blocks(js_code: str) -> list[str]:
    blocks = []
    stack = []
    start = None
    i = 0
    while i < len(js_code):
        if js_code[i:i+8].startswith('function'):
            if not stack:
                start = i
        if js_code[i] == '{':
            stack.append(i)
        elif js_code[i] == '}':
            if stack:
                stack.pop()
                if not stack and start is not None:
                    blocks.append(js_code[start:i+1].strip())
                    start = None
        i += 1
    return blocks

print(f".js file count: {len(js_files)} \nfolder: {MASTER_JS_PATH}")


# 建立資料集
all_functions = []

for js_file in js_files:
    try:
        js_code = read_text_file_safely(str(js_file))
        functions = extract_function_blocks(js_code)
        for func in functions:
            # d = {
            #     "file": js_file.name,
            #     "function": func
            # }
            # all_functions.append(json.dumps(d, ensure_ascii=False, indent=2))
            all_functions.append({"file": js_file.name, "function": func})
    except Exception as e:
        print(f"讀取或解析失敗: {js_file}，原因: {e}")

df = pd.DataFrame(all_functions)
grouped_all_functions = df.groupby("file")["function"].apply(list).reset_index()

# 只取第一個檔案的function實驗
print(grouped_all_functions.iloc[0]["function"])


.js file count: 88 
folder: C:\users\tuf\documents\cloudysys_nickfury\dcmsln\dcmsln_201812\Master\PageJS\Query
['function InitPage(){document.all.TXT_ProdID.focus()}', 'function checkProdInfoHead(){ if((event.keyCode == 13) ||(event.keyCode == 9) ){//alert("enter checkpageinfo");var oCtl;var sLineNo;var  sResult;oCtl=document.all.TXT_ProdID;if (sTrim(oCtl.value)==""){oCtl=document.all.TXT_ProdName;oCtl.value=""event.keyCode = 9;return true;}sResult=checkProdInfo(escape(sTrim(oCtl.value)));if ( sResult.PRODID=="" ){//没有取到数据说明输入的产品代码不存在（不需要自己写报错信息）try{//清空原来的产品名稱等数据oCtl=document.all.TXT_ProdName;oCtl.value="";oCtl=document.all.TXT_ProdID;oCtl.value="";}catch(e){}event.returnValue = false;return false;      }else{//开始将取到的产品名稱等数据填到画面控件中oCtl=document.all.TXT_ProdName;oCtl.value=sResult.PRODNAMECN;oCtl=document.all.TXT_ProdID;oCtl.value=sResult.PRODID;}event.keyCode = 9;return true;  }}', 'function openProdWindow(){var result = popQuickWindow(escape(document.all.TXT_ProdID.value),POPWINDOW_P

In [ ]:
# RAG

# 載入向量資料庫
def load_vector_db() -> FAISS:
    return FAISS.load_local(VECTOR_STORE_PATH, OllamaEmbeddings(model=EMBEDDING_MODEL), allow_dangerous_deserialization=True)

vector_db = load_vector_db()

# 查詢 function，可包成工具
def search_codebase(query: str, top_k: int = 3) -> str:
    results: List[LangDocument] = vector_db.similarity_search(query, k=top_k)
    return "\n\n".join([f"[{i+1}] From `{doc.metadata['source']}`\n{doc.page_content}" for i, doc in enumerate(results)])



C:\Users\TUF\AppData\Local\Temp\ipykernel_30636\3423936312.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  return FAISS.load_local(VECTOR_STORE_PATH, OllamaEmbeddings(model=EMBEDDING_MODEL), allow_dangerous_deserialization=True)


#  Agent Coding Example

In [ ]:
# autogen setting
local_ollama_port = "http://localhost:11434"
config_list = [
    {
        "model": "qwen3:8b",
        "api_type": "ollama",
        "stream": False,
        "base_url": local_ollama_port,
    }
]

In [57]:
# Setting up the code executor
workdir = Path("coding")
workdir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

# Setting up the agents

# The UserProxyAgent will execute the code that the AssistantAgent provides
# Perhaps abandone
user_proxy_agent = UserProxyAgent(
    name="User",
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: any(
        key in msg.get("content", "") for key in ["FINISH", "Final Answer"]
    ),
    human_input_mode="NEVER"
)

system_message = """You are a helpful AI assistant who writes code and the user
executes it. Solve tasks using your javascript coding skills.
In the following cases, suggest javascript code (in a javascript coding block) for the
user to execute. When using code, you must indicate the script type in the code block.
You only need to create one working sample.
Do not suggest incomplete code which requires users to modify it.
Don't use a code block if it's not intended to be executed by the user. Don't
include multiple code blocks in one response. Do not ask users to copy and
paste the result. Instead, use 'console.log' function for the output when relevant.
Check the execution result returned by the user.

If the result indicates there is an error, fix the error.

IMPORTANT: If it has executed successfully, ONLY output 'FINISH'."""

# The AssistantAgent, using the Ollama config, will take the coding request and return code
assistant_agent = AssistantAgent(
    name="Ollama Assistant",
    system_message=system_message,
    llm_config={"config_list": config_list},
)

In [58]:
# Start the chat, with the UserProxyAgent asking the AssistantAgent the message
chat_result = user_proxy_agent.initiate_chat(
    assistant_agent,
    message=f"Provide code to rewrite or improve the function: {grouped_all_functions.iloc[0]['function'][0]}",
)

User (to Ollama Assistant):

Provide code to rewrite or improve the function: function InitPage(){document.all.TXT_ProdID.focus()}

--------------------------------------------------------------------------------
Ollama Assistant (to User):

<think>
Okay, let's see. The user provided a function InitPage that focuses on an element with the ID TXT_ProdID using document.all. I need to rewrite or improve this function.

First, I remember that document.all is an older method, mainly used in Internet Explorer. Modern browsers don't support it, so using document.getElementById would be better. Also, the function is probably meant to focus on an input field, maybe a textbox.

So, the original code is function InitPage(){document.all.TXT_ProdID.focus()}. To improve this, I should replace document.all with document.getElementById. Also, maybe add a check to ensure the element exists before trying to focus, to prevent errors if the element isn't present.

Wait, but in JavaScript, if the element i

# Three Agents Coding Example (not in a group)

In [59]:
model_names = ["qwen3:8b", "gemma3:4b", "deepcoder:1.5b"]

In [66]:
# Shared system message
system_message = """You are a helpful AI assistant who writes code and the user
executes it. Solve tasks using your javascript coding skills.
In the following cases, suggest javascript code (in a javascript coding block) for the
user to execute. When using code, you must indicate the script type in the code block.
You only need to create one working sample.
Do not suggest incomplete code which requires users to modify it.
Don't use a code block if it's not intended to be executed by the user. Don't
include multiple code blocks in one response. Do not ask users to copy and
paste the result. Instead, use 'console.log' function for the output when relevant.
Check the execution result returned by the user. 

If the result indicates there is an error, fix the error.

IMPORTANT: If it has executed successfully, ONLY output 'FINISH'."""

In [67]:
# Create working directory and executor
workdir = Path("coding")
workdir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

In [68]:
# Create UserProxyAgent
user_proxy_agent = UserProxyAgent(
    name="User",
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: any(
        key in msg.get("content", "") for key in ["FINISH", "Final Answer"]
    ),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5
)

In [69]:
# Create AssistantAgents for each model
assistant_agents = []
for model in model_names:
   agent = AssistantAgent(
       name=f"Assistant_{model.replace(':', '_')}",
       system_message=system_message,
       llm_config={
           "config_list": [
               {
                   "model": model,
                   "api_type": "ollama",
                   "stream": False,
                   "base_url": local_ollama_port,
                   "temperature": 0.3
               }
            ]
        },
   )
   assistant_agents.append(agent)

In [ ]:
# Run the task with each assistant in sequence
task = f"Please rewrite the following legacy JavaScript function using modern JavaScript (ES6+), especially the old javascript code style. Function to modernize is ```{grouped_all_functions.iloc[0]['function'][0]}```"
print(task)

for assistant in assistant_agents:
    print(f"\n=== Running with {assistant.name} ===")
    result = user_proxy_agent.initiate_chat(
        assistant,
        message=task
    )

Please rewrite the following legacy JavaScript function using modern JavaScript (ES6+). Function to modernize is ```function InitPage(){document.all.TXT_ProdID.focus()}```

=== Running with Assistant_qwen3_8b ===
User (to Assistant_qwen3_8b):

Please rewrite the following legacy JavaScript function using modern JavaScript (ES6+). Function to modernize is ```function InitPage(){document.all.TXT_ProdID.focus()}```

--------------------------------------------------------------------------------
Assistant_qwen3_8b (to User):

<think>
Okay, let's see. The user wants to modernize the legacy JavaScript function InitPage. The original code uses document.all.TXT_ProdID.focus(). 

First, I remember that document.all is a collection of elements, and it's considered outdated. Modern JavaScript uses document.getElementById or other methods. Also, the function is named InitPage, which might be a good name to keep.

So, the original code is trying to focus on an element with the ID TXT_ProdID. In mo

qwen3_8b、gemma3_4b 表現還不錯，gemma3_4b更穩定一點

# Agent Coding Example (in a group)

In [ ]:
# Create chat group for conversation between agents
gemma3_4b_config = {
    "config_list": [
        {
            "model": "gemma3:4b",
            "api_type": "ollama",
            "stream": False,
            "base_url": local_ollama_port,
            "temperature": 0.3
        }
    ]
}
conversable_user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

coder1 = ConversableAgent(
    'Javascript Rewriter',
    llm_config=gemma3_4b_config,
    system_message=system_message,
)

coder2 = ConversableAgent(
    'Javascript Reviewer',
    llm_config=gemma3_4b_config,
    system_message=system_message,
)

# register_function(
#     search_codebase,
#     caller=coder1,
#     executor=conversable_user_proxy, 
#     name="search_codebase",
#     description="search_codebase searches codebase for related functions using a query. Useful for code reference or reuse."
# )

# register_function(
#     search_codebase,
#     caller=coder2,
#     executor=conversable_user_proxy, 
#     name="search_codebase",
#     description="search_codebase searches codebase for related functions using a query. Useful for code reference or reuse."
# )


def select_speaker(last_speaker, group_chat):
    if last_speaker is coder1:
        return coder2
    return coder1


group_chat = GroupChat(
    agents=[coder1, coder2],
    speaker_selection_method=select_speaker,
    messages=[],
    max_round=4,
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=gemma3_4b_config,
)

result = group_chat_manager.initiate_chat(
    coder1,
    message=task
)

chat_manager (to Javascript Rewriter):

Please rewrite the following legacy JavaScript function using modern JavaScript (ES6+). Function to modernize is ```function openProdWindow(){var result = popQuickWindow(escape(document.all.TXT_ProdID.value),POPWINDOW_PRODUCT);if( result!=null){if(result[0].PRODID!=null){if(result[0].PRODID!=""){document.all.TXT_ProdID.value = result[0].PRODID;document.all.TXT_ProdName.value = result[0].PRODNAMECN;}}}}```

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Javascript Rewriter (to chat_manager):

```javascript
function openProdWindow() {
  const productID = document.all.TXT_ProdID.value;
  const popupWindowProduct = "PRODUCT";

  const result = popQuickWindow(encodeURIComponent(productID), popupWindowProduct);

  if (result && Array.isArray(result)) {
    if (result.length > 0 && result[0]) {
      if (result[0].PRODID !== null && result[0].PRODID !== "") {
        document.all.TXT_ProdID

# 測試 Agent coding with multi task (實戰)

In [81]:
all_rewrited_function = []
print(f"測試單一個檔案 {grouped_all_functions.iloc[0]['file']}")

for function in grouped_all_functions.iloc[0]['function']:
    task = f"""You are tasked with modernizing the following legacy JavaScript function. 
    Please rewrite it using modern JavaScript (ES6+) standards, such as `document.all`/`document.getElementById` or `attachEvent`/`addEventListener`.

    Function to modernize:
    ```{function}```
    """
    group_chat = GroupChat(
        agents=[coder1, coder2],
        speaker_selection_method=select_speaker,
        messages=[],
        max_round=4,
    )
    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=gemma3_4b_config,
    )
    result = group_chat_manager.initiate_chat(
        coder1,
        message=task
    )

    all_agent_replies = [msg["content"]
                         for msg in result.chat_history if msg["role"] == "assistant"]
    all_rewrited_function.extend(all_agent_replies)


print(all_rewrited_function)

測試單一個檔案 GOF100S.js
chat_manager (to Javascript Rewriter):

You are tasked with modernizing the following legacy JavaScript function. 
    Please rewrite it using modern JavaScript (ES6+) standards, such as `document.all`/`document.getElementById` or `attachEvent`/`addEventListener`.

    Function to modernize:
    ```function InitPage(){document.all.TXT_ProdID.focus()}```
    

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Javascript Rewriter (to chat_manager):

```javascript
function InitPage() {
  const prodIdElement = document.getElementById('TXT_ProdID');
  if (prodIdElement) {
    prodIdElement.focus();
  }
}
```

--------------------------------------------------------------------------------

Next speaker: Javascript Reviewer


>>>>>>>> USING AUTO REPLY...
Javascript Reviewer (to chat_manager):

```javascript
function InitPage() {
  const prodIdElement = document.getElementById('TXT_ProdID');
  if (prodIdElement) {

In [82]:

# 指定輸出檔案路徑
output_file_path = f"./coding/{grouped_all_functions.iloc[0]['file']}"

# 過濾出包含 JavaScript function 的內容
cleaned_functions = []
function_pattern = re.compile(r"```javascript(.*?)```", re.DOTALL)

for idx, content in enumerate(all_rewrited_function, start=1):
    matches = function_pattern.findall(content)
    if matches:
        for match in matches:
            cleaned_functions.append(f"// Function #{len(cleaned_functions) + 1}\n{match.strip()}\n")

# 寫入檔案
with open(output_file_path, "w", encoding="utf-8") as f:
    f.writelines("\n".join(cleaned_functions))

print(f"已清除非程式碼內容並輸出 {len(cleaned_functions)} 個函式到 {output_file_path}")

已清除非程式碼內容並輸出 4 個函式到 ./coding/GOF100S.js
